##### This module runs the ECG algorithm, tailored to the TimeWak datasets


In [1]:
import numpy as np
import pandas as pd

import ECG.ECG_parameters as param
from ECG.ECG_fragile import SignalProcessing, WatermarkGenerator, FragileWatermark, SignalAnalysis2
from ECG.ECG_robust import Preprocessing, WatermarkEmbedding, SignalAnalysis
from utils import get_mae, normalize_dataset

In [8]:
"""From TimeWak. load the real and fake data"""
iterations = 5

original_data = pd.read_csv('datasets/energy_data.csv', 
# original_data = pd.read_csv('datasets/ETTh.csv', 
                            parse_dates=[0], 
                            date_format= '%d-%m-%y %H:%M')


In [9]:
"""Functions for Robust and Fragile embedding for 1 timeseries. The input timeseries is normalized!
NOTE the input to fragile watermarking is the ROBUSTLY WATERMARKED signal"""


def run_robust_ECG_for_1_column(column_number):
    """Run the ECG watermark for 1 column. NOTE we normalize the signal y-vals before watermarking"""
    # n_timesteps = math.floor(subsequence_length * param.num_subsequences) # from paper
    n_timesteps = len(original_data.iloc[:, 0]) # how many x-values
    input_signal= normalize_dataset(original_data.iloc[:, column_number].values)

    binary_ssn       = Preprocessing.convert_ssn_to_binary(param.user_ssn)
    binary_ssn_split = Preprocessing.split_and_pad_binary_ssn(binary_ssn, binary_ssn_chunk_len = 4)
    ssn_with_hamming = Preprocessing.apply_hamming_to_all_ssn_chunks(binary_ssn_split)
    subsequence_length = param.subsequence_len_factor * len(ssn_with_hamming) # m = 3*l (from paper)
    watermark_sequence = WatermarkEmbedding._turn_watermark_to_nonbinary_sequence(ssn_with_hamming)
    ecg_subsequences   = WatermarkEmbedding._split_signal_to_subsequences(input_signal, subsequence_length, n_timesteps)
    watermarked_signal = WatermarkEmbedding.get_watermarked_subsequences(ecg_subsequences, watermark_sequence)

    return input_signal, watermarked_signal

def run_fragile_ECG_for_1_column(watermarked_signal):
    """The input to this is the ROBUSTLY WATERMARKED signal (output of the robust watermark algo)"""

    shifted_signal, min_value       = SignalProcessing.shift_signal_up_to_remove_negative_values(watermarked_signal)
    scaled_signal                   = SignalProcessing.scale_signal_and_remove_decimals(shifted_signal, param.ECG_SCALE_FACTOR)
    scaled_signal_no_lsb            = SignalProcessing.remove_lsb_from_each_element_in_signal(scaled_signal)
    segments_list, num_segments_in_signal= WatermarkGenerator.split_signal_to_heartbeat_segments(scaled_signal_no_lsb)
    window_indices_for_all_segments = WatermarkGenerator.get_window_indices_for_all_segments(segments_list, param.SEED_K)
    segment_hashes                  = FragileWatermark.compute_segment_power_hashes(scaled_signal_no_lsb, window_indices_for_all_segments, num_segments_in_signal)
    quantized_segment_hashes        = FragileWatermark.quantize_hash_values_for_all_segments(segment_hashes, param.BIT_LENGTH)
    seeded_hash_segments            = FragileWatermark.prepend_seed_to_every_hash(quantized_segment_hashes, param.SEED_K, param.BIT_LENGTH)
    watermarks_for_all_segments     = FragileWatermark.convert_hash_to_int_and_generate_watermark(segments_list, seeded_hash_segments)
    watermarked_segments            = FragileWatermark.apply_lsb_watermark_to_ecg_segments(scaled_signal_no_lsb, segments_list, watermarks_for_all_segments)
    watermarked_signal              = FragileWatermark.concat_watermarked_segments(watermarked_segments)
    watermarked_signal_unscaled     = SignalProcessing.unscale_signal(watermarked_signal, param.ECG_SCALE_FACTOR)
    watermarked_signal_unshifted    = SignalProcessing.unshift_signal_back_to_original(watermarked_signal_unscaled, min_value)

    return watermarked_signal_unshifted


In [10]:
"""Run the single-timeseries Robust+fragile embedding algo on ALL timeseries (except for x-axis)"""

num_columns = original_data.shape[1]

mae_list = []
for col_num in range(1, num_columns): # start at col 1 to ignore x-axis
    input_signal, watermarked_signal = run_robust_ECG_for_1_column(col_num)
    watermarked_signal_unshifted     = run_fragile_ECG_for_1_column(watermarked_signal)

    mae = get_mae(input_signal, watermarked_signal_unshifted)
    mae_list.append(mae)

print(f"ECG robust+fragile MAE: avg={np.mean(mae_list)},\n std={np.std(mae_list)}")

should_we_plot = 0
SignalAnalysis.plot_robust_results(should_we_plot, input_signal, watermarked_signal)

should_we_plot = 0
SignalAnalysis2.plot_fragile_results(should_we_plot, watermarked_signal_unshifted)

/Users/fouadabiad/Projects/watermarking/myenv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/fouadabiad/Projects/watermarking/myenv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: cannot convert float NaN to integer